# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [4]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [5]:
# Calculate the total number of unique schools
unique_schools = school_data_complete['school_name'].unique()
school_count = len(unique_schools)
school_count

15

In [6]:
# Calculate the total number of students
student_count = school_data_complete["student_name"].count()
student_count

39170

In [7]:
# Calculate the total budget
total_budget = school_data["budget"].sum()
total_budget

24649428

In [8]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [9]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [10]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
students_passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
passing_math_count = students_passing_math["Student ID"].count()
passing_math_percentage = (passing_math_count / student_count) * 100
passing_math_percentage

74.9808526933878

In [11]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
students_passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
passing_reading_count = students_passing_reading["Student ID"].count()
passing_reading_percentage = (passing_reading_count / student_count) * 100
passing_reading_percentage

85.80546336482001

In [12]:
# Use the following to calculate the percentage of students that passed math and reading

overall_passing_rate = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)]['Student ID'].count()
overall_passing_rate

25528

In [13]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": school_count,
    "Total Students": f"{student_count:,}",
    "Total Budget": f"${total_budget:,.2f}",
    "Average Math Score": f"{average_math_score:.6f}",
    "Average Reading Score": f"{average_reading_score:.5f}",
    "% Passing Math": f"{passing_math_percentage:.6f}",
    "% Passing Reading": f"{passing_reading_percentage:.6f}",
    "% Overall Passing": f"{overall_passing_rate:.6f}"
    
    
    
}, index=[0])



# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,25528.000000


## School Summary

In [14]:
#Group by school name
school_name = school_data_complete.set_index('school_name').groupby(['school_name'])

In [15]:
# Use the code provided to select the school type
school_type = school_data.set_index(["school_name"])["type"]

In [16]:
# Calculate the total student count
per_school_counts = school_name['Student ID'].count()

In [17]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [18]:
# Calculate the average test scores
per_school_math = school_name['math_score'].mean()
per_school_reading = school_name['reading_score'].mean()

In [27]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
school_passing_math_count = school_passing_math["School ID"].count()


#students_passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
#passing_math_count = students_passing_math["Student ID"].count()



In [29]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
school_passing_reading_count = school_passing_reading["School ID"].count()

In [21]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

In [30]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [33]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    
    "School Type": school_type,
    "Total Students": per_school_counts,
    "Per Student Budget": per_school_capita,
    "Total School Budget": per_school_budget,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    '% Passing Math': per_school_passing_math,
    '% Passing Reading': per_school_passing_reading,
    "% Overall Passing": overall_passing_rate
    
    
    
    
    
})

 



#munging
per_school_summary = per_school_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          '% Overall Passing']]


#formatting
per_school_summary.style.format({'Total Students': '{:}',
                          "Total School Budget": "${:,.2f}",
                          "Per Student Budget": "${:.2f}",
                          'Average Math Score': "{:6f}", 
                          'Average Reading Score': "{:6f}", 
                          "% Passing Math": "{:6f}", 
                          "% Passing Reading": "{:6f}"})








# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [36]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.

top_schools = per_school_summary.sort_values("% Overall Passing", ascending = False)
top_schools.head().style.format({'Total Students': '{:}',
                           "Total School Budget": "${:,.2f}", 
                           "Per Student Budget": "${:.2f}", 
                           "% Passing Math": "{:6f}", 
                           "% Passing Reading": "{:6f}", 
                           "% Overall Passing": "{:6f}"})





,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [37]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.

bottom_schools = top_schools.tail()
bottom_schools = bottom_schools.sort_values('% Overall Passing')
bottom_schools.style.format({'Total Students': '{: }', 
                       "Total School Budget": "${:,.2f}", 
                       "Per Student Budget": "${:.2f}", 
                       "% Passing Math": "{:6f}", 
                       "% Passing Reading": "{:6f}", 
                       "% Overall Passing": "{:6f}"})





,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [40]:
# Use the code provided to separate the data by grade. Group by "school_name" and take the mean of each.
ninth_graders_math = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_graders_math = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_graders_math = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_graders_math = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()




# Use the code to select only the `math_score and combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
        "9th": ninth_graders_math,
        "10th": tenth_graders_math,
        "11th": eleventh_graders_math,
        "12th": twelfth_graders_math

})

math_scores_by_grade = math_scores_by_grade[['9th', '10th', '11th', '12th']]
math_scores_by_grade.index.name = "School Name"

# Display the DataFrame
math_scores_by_grade.style.format({'9th': '{:.6f}', 
                          "10th": '{:.6f}', 
                          "11th": "{:.6f}", 
                          "12th": "{:.6f}"})






,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [41]:
# Use the code provided to separate the data by grade. # Group by "school_name" and take the mean of each.
ninth_graders_reading = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_graders_reading = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_graders_reading = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_graders_reading = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()




# Use the code to select only the reading_score and combine each of the scores above into single DataFrame called reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
        "9th": ninth_graders_reading,
        "10th": tenth_graders_reading,
        "11th": eleventh_graders_reading,
        "12th": twelfth_graders_reading

})

reading_scores_by_grade = reading_scores_by_grade[['9th', '10th', '11th', '12th']]
reading_scores_by_grade.index.name = "School Name"

# Display the DataFrame
reading_scores_by_grade.style.format({'9th': '{:.6f}', 
                          "10th": '{:.6f}', 
                          "11th": "{:.6f}", 
                          "12th": "{:.6f}"})


,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [63]:
# Establish the bins and use `pd.cut` to categorize spending based on the bins.
the_bins = [0, 585, 630, 645, 680]
label = ["<$585", "$585-630", "$630-645", "$645-680"]
school_data_complete['spending_bins'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], the_bins, labels = label)

#group the schools  by spending
group_by_spending = school_data_complete.groupby('spending_bins')


# Calculate averages for the desired columns
spending_math_score = group_by_spending['math_score'].mean()
spending_reading_score = group_by_spending['reading_score'].mean()
spending_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/group_by_spending['Student ID'].count()*100
spending_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/group_by_spending['Student ID'].count()*100
overall_passing_spending = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/group_by_spending['Student ID'].count()*100
    
            
scores_by_school_spending = pd.DataFrame({
    "Average Math Score": spending_math_score,
    "Average Reading Score": spending_reading_score,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
            
})
            

scores_by_school_spending = scores_by_school_spending[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]]

scores_by_school_spending.index.name = "Per Student Budget"


#Display the DataFrame
scores_by_school_spending.style.format({'Average Math Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Math': '{:.2f}', 
                              '% Passing Reading':'{:.2f}', 
                              '% Overall Passing': '{:.2f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Budget,,,,,
<$585,83.36,83.96,93.70,96.69,90.64
$585-630,79.98,82.31,79.11,88.51,70.94
$630-645,77.82,81.30,70.62,82.60,58.84
$645-680,77.05,81.01,66.23,81.11,53.53


## Scores by School Size

In [68]:
# Establish the bins
the_bins = [0, 1000, 1999,5000]
label = ["Small (<1000)", "Medium (1000-2000)" , "Large (2000-5000)"]
school_data_complete['size_bins'] = pd.cut(school_data_complete['size'], the_bins, labels = label)

#group by size
group_by_size = school_data_complete.groupby('size_bins')






# Calculate averages for the desired columns
size_math_scores = group_by_size['math_score'].mean()
size_reading_scores = group_by_size['math_score'].mean()
size_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/group_by_size['Student ID'].count()*100
size_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/group_by_size['Student ID'].count()*100
size_overall_passing = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/group_by_size['Student ID'].count()*100

            
       
scores_by_school_size = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    '% Passing Math': size_passing_math,
    '% Passing Reading': size_passing_reading,
    '% Overall Passing': size_overall_passing
            
})
            

scores_by_school_size = scores_by_school_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing'
]]

scores_by_school_size.index.name = "Total Students"
scores_by_school_size = scores_by_school_size.reindex(label)

#Display the DataFrame
scores_by_school_size.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6f}', 
                              '% Passing Reading':'{:.6f}', 
                              '% Overall Passing': '{:.6f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Total Students,,,,,
Small (<1000),83.828654,83.828654,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.372682,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,77.477597,68.652380,82.125158,56.574046


## Scores by School Type

In [71]:
# group by type of school
group_by_school_type = school_data_complete.groupby("type")



# Calculate averages for the desired columns
type_math_score = group_by_school_type['math_score'].mean()
type_reading_score = group_by_school_type['math_score'].mean()
type_math_percent = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/group_by_school_type['Student ID'].count()*100
type_read_percent = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/group_by_school_type['Student ID'].count()*100
type_overall_passing = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/group_by_school_type['Student ID'].count()*100

           
scores_by_school_type = pd.DataFrame({
    "Average Math Score": type_math_score,
    "Average Reading Score": type_reading_score,
    '% Passing Math': type_math_percent,
    '% Passing Reading': type_read_percent,
    "% Overall Passing": type_overall_passing})
    

scores_by_school_type = scores_by_school_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing"
]]
scores_by_school_type.index.name = "Type of School"


#Display the DataFrame
scores_by_school_type.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6f}', 
                              '% Passing Reading':'{:.6f}', 
                              '% Overall Passing': '{:.6f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Type of School,,,,,
Charter,83.406183,83.406183,93.701821,96.645891,90.560932
District,76.987026,76.987026,66.518387,80.905249,53.695878
